# Rough Calculation of Capillary Force Along Major Axis for All Datasets
  
Written by Kara Hartig 

Edited by Maya Faaborg and Ahmed Sherif

Note on distances, displacement vs deflection:  
In this experiment, when fiber is deflected, it ends up partway between the center of the channel and its neutral (undeflected) position; this is where the restoration force matches the capillary force. There are therefore three distances in this experiment:  
d: the total displacement of the detector (distance between neutral position and center of channel)  
delta: the deflection of the fiber (distance between neutral position and tip of deflected fiber)  
x: the displacement of the fiber from center (distance between center of channel and tip of deflected fiber)  

We control d through the constant speed of the motor stage, we measure delta with the detector (and analysis below), so we can find x using x = d - delta. This will be done when we produce the plot of force vs displacement from center.

In [ ]:
# Packages
import numpy as np
from scipy import ndimage
import os
from skimage import io
import csv
import matplotlib.pyplot as plt
from matplotlib import patches
plt.rcParams.update({'font.size': 20})
%matplotlib inline

In [ ]:
# Functions
def COM_stack(data,mask_threshold=50.):
    """
    Assuming data indices are arranged [frames, x, y], applies a mask (intensity cut-off set by mask_threshold)
    and finds center of mass in x,y format for each masked frame.
    """
    com = np.zeros((len(data),2))
    for ind,frame in enumerate(data):
        masked = 1.0*frame[:,:]
        masked[masked<mask_threshold] = 0.
        masked[masked>=mask_threshold] = 100.
        com[ind,1],com[ind,0] = ndimage.measurements.center_of_mass(masked)  # COM given in [row,column], row=y and column=x
    return com

def travel_line(x,com):
    """
    Given list of x values and list of center-of-mass coordinates, calculates y(x) between initial and final COM points
    """
    x1,y1 = com[0]
    x2,y2 = com[-1]
    y = (y2-y1)/(x2-x1)*(x-x1) + y1
    return y

def fiber_deflection_mag(com,reference_position,scale=1.0):
    """
    Given list of center-of-mass coordinates and a reference position (x0,y0), calculate magnitude of deflection
    (multiplied by scale) from reference
    """
    x0,y0 = reference_position
    y = [scale*np.sqrt((x-x0)**2+(y-y0)**2) for x,y in com]
    return y

def fiber_deflection_vector(com,reference_position,scale=1.0):
    """
    Given list of center-of-mass coordinates and a reference position (x0,y0), calculate deflection (multiplied by scale)
    from reference as an x,y pair
    """
    x0,y0 = reference_position
    x = [scale*(x-x0) for x,y in com]
    y = [scale*(y-y0) for x,y in com]
    return x,y

def force_from_deflection(deflection):
    """
    Given deflection from fiber_deflection_mag, calculates restoration force of fiber
    """
    r = 110.0e-6  # m; radius of optical fiber
    E = 7.46e10  # Young's modulus (from Miles's Draper calculations)
    L = 12.3e-2  # m; Length of fiber from clamp to tip; in lab notebook
    F = [(3*np.pi*E*r**4*delta)/(4*L**3) for delta in deflection]
    return F

In [ ]:
# Data locations
major_axis_dir = "⁨manoharan_lab⁩/⁨khartig⁩/⁨Data⁩/Direct_capillary/⁩major_axis/"
dict_map = ['a','b','c','d','e','f']
color_scheme = {'all_same':'#404040', 'sample':'#F29724', 'neutral':'#B80D48'}
lw = 3.0  # universal linewidth
data_folders = {'a': "2018-11-27-afternoon/center_to_leftedge/",
                'b': "2018-11-27-afternoon/fullsweep_left/",
                'c': "2018-11-27-afternoon/fullsweep_right/",
                'd': "2018-11-28/center_to_leftedge/",
                'e': "2018-11-28/fullsweep_left/",
                'f': "2018-11-28/fullsweep_right/"}
file_prefixes = {'a': 'data_one',
                 'b': 'data_three',
                 'c': 'data_two',
                 'd': 'data_one',
                 'e': 'data_three',
                 'f': 'data_two'}
file_suffixes = {'a': [".tif"],
                 'b': [".tif","_0.tif"],
                 'c': [".tif","_0.tif"],
                 'd': [".tif"],
                 'e': [".tif","_0.tif"],
                 'f': [".tif","_0.tif"]}
file_paths = {'placeholderkey':'placeholdervalue'}
for key in dict_map:
    file_path = []
    for suffix in file_suffixes[key]:
        file_path.append(os.path.normpath(major_axis_dir + data_folders[key] + file_prefixes[key] + suffix))
    file_paths[key] = file_path
del file_paths['placeholderkey']
start_frame_indices = {'a': 19,'b': 19,'c': 19,'d': 20,'e': 21,'f': 19}

In [ ]:
# Read in images (red channel only) and calculate COM
#imstack = {'placeholderkey':'placeholdervalue'}
num_frames = {'placeholderkey':'placeholdervalue'}
com = {'placeholderkey':'placeholdervalue'}
for key in dict_map:
    frames = np.concatenate([io.imread(x)[:,:,:,0] for x in file_paths[key]])[start_frame_indices[key]:]
    #imstack[key] = frames
    num_frames[key] = len(frames)
    com[key] = COM_stack(frames,mask_threshold=100)
    print(key, frames.shape, len(com[key]))
#del imstack['placeholderkey']
del num_frames['placeholderkey']
del com['placeholderkey']

In [ ]:
# Motor and Camera Settings
#    Motor Settings
motor_startstopspeed = {'a': (7.4,3.2,-0.1),
                        'b': (10.0,3.2,-0.1),
                        'c': (3.2,10.0,0.1),
                        'd': (7.4,3.4,-0.1),
                        'e': (11.0,3.4,-0.1),
                        'f': (3.4,11.0,0.1)}
motor_at_neutral = 7.4  # happens to be the same for all major axis datasets

#    Camera Settings
fps = 5.  # same for every dataset
distance_per_pixel = 0.00078  # mm per pixel; found using ImageJ on calibration_3

#    Timestamp
times = {'placeholderkey':'placeholdervalue'}
for key in dict_map:
    times[key] = np.arange(start=0., stop=num_frames[key]/fps, step=1/fps)
del times['placeholderkey']

#    Device and Meniscus Position
#       padded the end with final position to account for motor stopping before frames do
motor_position = {'placeholderkey':'placeholdervalue'}
meniscus_position = {'placeholderkey':'placeholdervalue'}
for key in dict_map:
    initial, final, speed = motor_startstopspeed[key]
    motor = np.arange(start=initial,stop=final,step=speed/fps)
    motor_position[key] = np.pad(motor,pad_width=(0,num_frames[key]-len(motor)),mode='constant',constant_values=final)
    meniscus_position[key] = motor_position[key] - motor_at_neutral
del motor_position['placeholderkey']
del meniscus_position['placeholderkey']

In [ ]:
# Find index of neutral position
threshold = 0.00001
neutral_index = {'placeholderkey':'placeholdervalue'}
for key in dict_map:
    x = [ind for ind,val in enumerate(motor_position[key]) if abs(val - motor_at_neutral) < threshold]
    if len(x) > 1:
        raise ValueError('Found more than one index ({}) corresponding to neutral position for key {}'.format(x,key))
    elif len(x) == 0:
        raise ValueError('Did not find index corresponding to neutral position for key {}'.format(key))
    else:
        neutral_index[key] = x.copy()[0]
del neutral_index['placeholderkey']

In [ ]:
# Plot COMs colored by time
for key in dict_map:
    com_x,com_y = zip(*com[key])
    plt.figure()
    plt.scatter(com_x,com_y,c=times[key])
    plt.colorbar()
    plt.scatter(com_x[neutral_index[key]],com_y[neutral_index[key]],c=color_scheme['neutral'],s=200,marker='x')
    plt.xlim(0,1280)
    plt.ylim(0,1024)
    plt.title('Center-of-Mass over time')
    #savename = 'Group Mtg Presentation/figures_from_code/majoraxis_COM_dataset-' + key + '.png'
    #plt.savefig(savename)

In [ ]:
# Calculate and plot fiber deflection
#    also calculates signed deflection: negative at times before neutral (for calculating displacement from center)
deflection = {'placeholderkey':'placeholdervalue'}
signed_deflection = {'placeholderkey':'placeholdervalue'}
for key in dict_map:
    com_x,com_y = zip(*com[key])
    n_ind = neutral_index[key]
    deflection[key] = fiber_deflection_mag(com[key],(com_x[n_ind],com_y[n_ind]),scale=distance_per_pixel)
    with_sign = []
    for ind,d in enumerate(deflection[key]):
        if ind < n_ind:
            with_sign.append(-d)
        else:
            with_sign.append(d)
    signed_deflection[key] = with_sign
del deflection['placeholderkey']
del signed_deflection['placeholderkey']

for key in dict_map:
    plt.figure()
    plt.plot(times[key],deflection[key],c=color_scheme['all_same'],linewidth=lw)
    plt.axvline(x=times[key][neutral_index[key]],c=color_scheme['neutral'])
    plt.title('Deflection over time')
    plt.xlabel('Time (s)')
    plt.ylabel('Deflection (mm)')
    #savename = 'Group Mtg Presentation/figures_from_code/majoraxis_deflection_dataset-' + key + '.png'
    #plt.savefig(savename)

In [ ]:
# Plot all fiber deflections centered on neutral
relative_times = {'placeholderkey':'placeholdervalue'}
for key in dict_map:
    relative_times[key] = times[key] - times[key][neutral_index[key]]
del relative_times['placeholderkey']

# Just sample dataset 'a'
plt.figure()
plt.axvline(x=0,c=color_scheme['neutral'])
plt.plot(relative_times['a'],deflection['a'],c=color_scheme['sample'],linewidth=lw)
plt.xlim(-50,50)
plt.ylim(0,0.7)
plt.title('Deflection from Neutral vs Time')
plt.xlabel('Time since Neutral (s)')
plt.ylabel('Deflection (mm)')
#savename = 'Group Mtg Presentation/figures_from_code/majoraxis_deflection_dataset-a.png'
#plt.savefig(savename)

# Just jump dataset 'e'
plt.figure()
plt.axvline(x=0,c=color_scheme['neutral'])
plt.plot(relative_times['e'],deflection['e'],c=color_scheme['all_same'],linewidth=lw)
plt.xlim(-50,50)
plt.ylim(0,0.7)
plt.title('Deflection from Neutral vs Time')
plt.xlabel('Time since Neutral (s)')
plt.ylabel('Deflection (mm)')
#savename = 'Group Mtg Presentation/figures_from_code/majoraxis_deflection_dataset-e.png'
#plt.savefig(savename)

# All datasets
plt.figure()
plt.axvline(x=0,c=color_scheme['neutral'])
plt.xlim(-50,50)
plt.ylim(0,0.7)
plt.title('Deflection from Neutral vs Time')
plt.xlabel('Time since Neutral (s)')
plt.ylabel('Deflection (mm)')
for key in dict_map:
    if key == 'a':
        plt.plot(relative_times[key],deflection[key],c=color_scheme['sample'],linewidth=lw,zorder=2)
    else:
        plt.plot(relative_times[key],deflection[key],c=color_scheme['all_same'],linewidth=lw,zorder=1)
#savename = 'Group Mtg Presentation/figures_from_code/majoraxis_deflection_alldatasets.png'
#plt.savefig(savename)

In [ ]:
# Plot fiber force centered on neutral
restoration_force = {'placeholderkey':'placeholdervalue'}
for key in dict_map:
    restoration_force[key] = force_from_deflection(0.01*np.ones(len(deflection[key]))*deflection[key])
del restoration_force['placeholderkey']

plt.figure()
plt.axvline(x=0,c=color_scheme['neutral'])
plt.title('Restoration Force vs Time')
plt.xlabel('Time since Neutral (s)')
plt.ylabel('Restoration Force (N)')
for key in dict_map:
    if key == 'a':
        plt.plot(relative_times[key],restoration_force[key],c=color_scheme['sample'],linewidth=lw,zorder=2)
    else:
        plt.plot(relative_times[key],restoration_force[key],c=color_scheme['all_same'],linewidth=lw,zorder=1)
#savename = 'Group Mtg Presentation/figures_from_code/majoraxis_force_alldatasets.png'
#plt.savefig(savename)

In [ ]:
# Calculate and plot fiber force vs displacement from center (x)
fiber_displacement = {'placeholderkey':'placeholdervalue'}
for key in dict_map:
    d = relative_times[key]*abs(motor_startstopspeed[key][2])
    delta = signed_deflection[key]
    fiber_displacement[key] = d - delta
del fiber_displacement['placeholderkey']

plt.figure()
plt.axvline(x=0,c=color_scheme['neutral'])
plt.title('Restoration Force vs Displacement from Center')
plt.xlabel('Fiber Displacement from Center of Channel (mm)')
#In order to show the plot in μN, I multiplied the restoration_force by 10^6 down below
plt.ylabel('Restoration Force (μN)')
for key in dict_map:
    if key == 'a':
        plt.plot(fiber_displacement[key],1000000*np.ones(len(restoration_force[key]))*restoration_force[key],c=color_scheme['sample'],linewidth=lw,zorder=2)
    else:
        plt.plot(fiber_displacement[key],1000000*np.ones(len(restoration_force[key]))*restoration_force[key],c=color_scheme['all_same'],linewidth=lw,zorder=1)

In [ ]:
# Save force vs displacement data and plot

# All Force vs Displacement plot
plt.figure()
plt.axvline(x=0,c=color_scheme['neutral'])
plt.title('Restoration Force vs Displacement from Center')
plt.xlabel('Fiber Displacement from Center of Channel (mm)')
plt.ylabel('Restoration Force (uN)')
for key in dict_map:
    plt.plot(fiber_displacement[key],1000000*np.ones(len(restoration_force[key]))*restoration_force[key],c=color_scheme['all_same'],linewidth=lw)
savename = '/mnt/groupshare/khartig/Data/Direct_capillary/major_axis/Final Force vs Displacement Plots/all_datasets_plot/majoraxis_forceVSdisplacement_alldatasets.png'
plt.savefig(savename)

# Individual deflection vs time since neutral datasets
#savepath = 'Final Force vs Displacement Plots/deflection_vs_time_since_neutral/'
#header = ['Time since neutral in seconds','Fiber deflection in mm']
#for key in dict_map:
#    savename = savepath + 'majoraxis_deflectionVStime_dataset-' + key + '.csv'
#    with open(savename, 'w') as file:
#        writer = csv.writer(file, lineterminator="\n")
#        writer.writerow(header)
#        for ind,time in enumerate(relative_times[key]):
#            writer.writerow([time, deflection[key][ind]])

# Individual force vs displacement from center datasets
savepath = '/mnt/groupshare/khartig/Data/Direct_capillary/major_axis/Final Force vs Displacement Plots/force_vs_displacement_from_center/'
header = ['Displacement of fiber from channel center in mm','Force in N']
for key in dict_map:
   savename = savepath + 'majoraxis_forceVSdisplacement_dataset-' + key + '.csv'
   with open(savename, 'w') as file:
       writer = csv.writer(file, lineterminator="\n")
       writer.writerow(header)
       for ind,disp in enumerate(fiber_displacement[key]):
           writer.writerow([disp, restoration_force[key][ind]])

In [ ]:
# Example of reading in and plotting force vs displacement
read_force = {'placeholderkey':'placeholdervalue'}
read_displacement = {'placeholderkey':'placeholdervalue'}

#set savepath
#savepath = '\\'
for key in dict_map:
    savename = 'majoraxis_forceVSdisplacement_dataset-' + key + '.csv'
    loop_force = []
    loop_displacement = []
    print('opening dataset ' + key)
    with open(savename, 'r') as file:
        reader = csv.reader(file)
        next(reader, None)  # skip header row
        for row in reader:
            a,b = row
            loop_displacement.append(float(a))
            loop_force.append(1e6*float(b))
    print('  finished reading dataset ' + key)
    read_force[key] = loop_force
    read_displacement[key] = loop_displacement
del read_force['placeholderkey']
del read_displacement['placeholderkey']

plt.figure()
plt.xlabel('Fiber displacement from center of channel (mm)')
plt.ylabel('Restoration force (μN)')
for key in dict_map:
    plt.plot(read_displacement[key],read_force[key],c=color_scheme['all_same'],linewidth=1)

plt.tight_layout()
#plt.savefig('test.pdf')

Taking averages of the experimental runs:

In [ ]:
# Example of reading in and plotting force vs displacement
read_force = {'placeholderkey':'placeholdervalue'}
read_displacement = {'placeholderkey':'placeholdervalue'}

for key in dict_map:
    savename = 'majoraxis_forceVSdisplacement_dataset-' + key + '.csv'
    loop_force = []
    loop_displacement = []
    print('opening dataset ' + key)
    with open(savename, 'r') as file:
        reader = csv.reader(file)
        next(reader, None)  # skip header row
        for row in reader:
            a,b = row
            loop_displacement.append(float(a))
            loop_force.append(1e6*float(b))
    print('  finished reading dataset ' + key)
    read_force[key] = loop_force
    read_force[key] = np.array(read_force[key])
    read_displacement[key] = loop_displacement
    read_displacement[key] = np.array(read_displacement[key])
del read_force['placeholderkey']
del read_displacement['placeholderkey']

plt.figure()
plt.xlabel('Fiber displacement from center of channel (mm)')
plt.ylabel('Restoration force (μN)')
for key in dict_map:
    plt.plot(read_displacement[key],read_force[key],linewidth=1, label=key)
    plt.legend()

plt.tight_layout()
#plt.savefig('test.pdf')

In [ ]:
#take arrays from -2 to 0 and 0 to -2 for each run , average each half, then plot

#can take each half of data like this:
c_disp_02 = read_displacement['c'][(read_displacement['c']<= 2) & (read_displacement['c'] >= 0)]
c_force_02 = read_force['c'][(read_displacement['c']<= 2) & (read_displacement['c'] >= 0)]


In [ ]:
# check for run c
plt.plot(read_displacement['c'],read_force['c'])
plt.plot(c_disp_02, c_force_02)

In [ ]:
full_runs = ['b','c','e','f'] #runs that have two halves

#take all runs from 0 to 2
force02 = {'placeholderkey':'placeholdervalue'}
displacement02 = {'placeholderkey':'placeholdervalue'}
for key in dict_map:
    displacement02[key] = read_displacement[key][(read_displacement[key]<= 2) & (read_displacement[key] >= 0)]
    force02[key] = read_force[key][(read_displacement[key]<= 2) & (read_displacement[key] >= 0)]
    
del displacement02['placeholderkey']
del force02['placeholderkey']    

#take the full runs from -2 to 0
force_neg = {'placeholderkey':'placeholdervalue'}
displacement_neg = {'placeholderkey':'placeholdervalue'}

for key in full_runs:
    displacement_neg[key] = read_displacement[key][(read_displacement[key]<= 0) & (read_displacement[key] >= -2)]
    force_neg[key] = read_force[key][(read_displacement[key]<= 0) & (read_displacement[key] >= -2)]
    
del force_neg['placeholderkey']
del displacement_neg['placeholderkey']

In [ ]:
#plot everything to check 
for key in dict_map:
    plt.plot(read_displacement[key],read_force[key],c='r',linewidth=1)
    plt.plot(displacement02[key],force02[key],c='b',linewidth=1)
    
for key in full_runs:
    plt.plot(displacement_neg[key], force_neg[key], c='g', linewidth=1)

In [ ]:
#need to interpolate in order to average runs

#interpolate  - check with one set of values
pos_disp = np.linspace(0, 2, 100)
neg_disp = np.linspace(-2, 0, 100)
test_interp = np.interp(pos_disp, displacement02['a'], force02['a'])

plt.plot(pos_disp, test_interp, 'ro')
plt.plot(displacement02['a'], force02['a'])

In [ ]:
# get interpolations for runs from 0 to 2 and -2 to 0 
num_points = 5000
pos_disp = np.linspace(0, 2, num_points)
neg_disp = np.linspace(-2, 0, num_points)

force02_interp = {'placeholderkey':'placeholdervalue'}
force_neg_interp = {'placeholderkey':'placeholdervalue'}

for key in dict_map:
    force02_interp[key] = np.interp(pos_disp, displacement02[key], force02[key])
    
for key in full_runs:
    force_neg_interp[key] = np.interp(neg_disp, displacement_neg[key], force_neg[key])

del force02_interp['placeholderkey']
del force_neg_interp['placeholderkey']



In [ ]:
#plot to check
for key in dict_map:
    plt.plot(read_displacement[key],read_force[key],c='r',linewidth=1)
    plt.plot(pos_disp,force02_interp[key],c='b',linewidth=1)
    
for key in full_runs:
    plt.plot(neg_disp, force_neg_interp[key], c='g', linewidth=1)

In [ ]:
#average the interpolated arrays on either side of 0

l_c = 2.7 #capillary length in mm

average_pos = (force02_interp['a']+force02_interp['b']+force02_interp['c']+force02_interp['d']+force02_interp['e']+force02_interp['f'])/6
average_neg = (force_neg_interp['b']+force_neg_interp['c']+force_neg_interp['e']+force_neg_interp['f'])/4

#plot everything and normalize x axis

for key in dict_map:
    plt.plot(displacement02[key]/l_c,force02[key],'k--',linewidth=.25)
    
for key in full_runs:
    plt.plot(displacement_neg[key]/l_c, force_neg[key], 'k--', linewidth=.25)
    

    
plt.plot(pos_disp/l_c, average_pos,c=color_scheme['all_same'],linewidth=1)
plt.plot(neg_disp/l_c,average_neg,c=color_scheme['all_same'],linewidth=1)



plt.xlabel('Fiber displacement from center of channel (normalized by $l_c$)')
plt.ylabel('Restoration force (μN)')

plt.tight_layout()
plt.savefig('force-disp-l_c.pdf')

In [ ]:
#make envelope for all the data (positive side)
allforces_pos = np.array([force02_interp['a'], force02_interp['b'], force02_interp['c'], force02_interp['d'], force02_interp['e'], force02_interp['f']])

minimum_forces_pos = np.min(allforces_pos, axis=0)
maximum_forces_pos = np.max(allforces_pos, axis =0)

for key in dict_map:
    plt.plot(displacement02[key]/l_c,force02[key],'k',linewidth=1)

plt.plot(pos_disp/l_c, maximum_forces_pos, 'b--')
plt.plot(pos_disp/l_c, minimum_forces_pos, 'r--')

In [ ]:
allforces_neg = np.array([force_neg_interp['b'],force_neg_interp['c'],force_neg_interp['e'],force_neg_interp['f']])

minimum_forces_neg = np.min(allforces_neg, axis=0)
maximum_forces_neg = np.max(allforces_neg, axis =0)

for key in full_runs:
    plt.plot(displacement_neg[key]/l_c, force_neg[key], 'k', linewidth=1)

plt.plot(neg_disp/l_c, maximum_forces_neg, 'b--')
plt.plot(neg_disp/l_c, minimum_forces_neg, 'r--')

In [ ]:
plt.fill_between(pos_disp/l_c, maximum_forces_pos, minimum_forces_pos, color='k', alpha=.1)
plt.fill_between(neg_disp/l_c, maximum_forces_neg, minimum_forces_neg, color='k', alpha=.1)


plt.plot(pos_disp/l_c, average_pos,c=color_scheme['all_same'],linewidth=1)
plt.plot(neg_disp/l_c,average_neg,c=color_scheme['all_same'],linewidth=1)



plt.xlabel('Fiber displacement from center of channel (normalized by $l_c$)')
plt.ylabel('Restoration force (μN)')

plt.tight_layout()
#plt.savefig('force-disp-l_c.pdf')

Using a boxcar average to smooth the average of the experiments:

In [ ]:
#append arrays to take boxcar more easily

full_disp = np.append(neg_disp, pos_disp)
full_avg = np.append(average_neg, average_pos)


In [ ]:
N = int(num_points/10)

plt.fill_between(pos_disp/l_c, maximum_forces_pos, minimum_forces_pos, color='k', alpha=.1)
plt.fill_between(neg_disp/l_c, maximum_forces_neg, minimum_forces_neg, color='k', alpha=.1)

#take average and plot
plt.plot(full_disp[int(N/2)-1:-int(N/2)]/l_c, np.convolve(full_avg, np.ones(N)/N, mode='valid'),c=color_scheme['all_same'],linewidth=1)



plt.xlabel('Fiber displacement from center of channel (normalized by $l_c$)')
plt.ylabel('Restoration force (μN)')


plt.tight_layout()
#plt.savefig('force-disp-l_c.pdf')


In [ ]:
gamma = 72 #mN / m surface tension
float_radius = 5e-3 #m 
#F_nondim =gamma*float_radius*2*np.pi*10**3 #um
F_nondim = gamma * l_c *10e-3 * 10e3

def norm_dist(x):
    return x/l_c

def norm_dist_inv(x):
    return l_c*x

def norm_force(y):
    return y/F_nondim

def norm_force_inv(y):
    return y*F_nondim

In [ ]:
fig, ax = plt.subplots(constrained_layout=True)

#dimensional
ax.fill_between(pos_disp, maximum_forces_pos, minimum_forces_pos, color='k', alpha=.1)
ax.fill_between(neg_disp, maximum_forces_neg, minimum_forces_neg, color='k', alpha=.1)

ax.plot(full_disp[int(N/2)-1:-int(N/2)], np.convolve(full_avg, np.ones(N)/N, mode='valid'),c=color_scheme['all_same'],linewidth=1)



ax.set_xlabel('Fiber displacement from center of channel (mm)')
ax.set_ylabel('Restoration force (μN)')

#non-dimensional 
secaxx = ax.secondary_xaxis('top', functions=(norm_dist, norm_dist_inv))
secaxx.set_xlabel('displacement/capillary length')

secaxy = ax.secondary_yaxis('right', functions=(norm_force, norm_force_inv))
secaxy.set_ylabel(r'Restoration force/$\gamma$ * capillary length')

#plt.savefig('force-disp-nondim.pdf')